# Quantum matched filtering for GW150914

In [ ]:
# Import necessary packages
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from misc.paper_plotting import plot_psi_1_paper, plot_psi_1_match_v_unmatch_paper
from misc.paper_plotting import plot_psi_2_paper, plot_psi_2_t_paper, plot_psi_2_k_paper
from misc.paper_plotting import plot_simulation
import quantum_matched_filter_functions as qmffn
import QMF_150914_simulation as qmfsim

import time, os

debug = False

# Set the random number generator seed 
np.random.seed(123)#int(time.time()))

In [ ]:
fontsize=24
ticksize=19
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
figsize=(10,8)

In [ ]:
tag='GW150814_example'
dtype='float64'

load_states=True
save_states=True

path='./outpath/'

In [ ]:
os.makedirs(path, exist_ok=True)

In [ ]:
# Set the threshold SNR
SNR_threshold = 12.
# Load the data (GW150914) and the corresponding psd
Data = np.load('data/signal.npy')
psd = np.load('data/psd.npy')

In [ ]:
fig = plt.figure(figsize=figsize)
Data_t = Data
plt.plot(np.arange(len(Data_t)), Data_t, lw=.2, color='black')
plt.xlabel(r'$t$ (s)', fontsize=fontsize)
plt.ylabel(r'$x(t)$ ($10^{-25}$)', fontsize=fontsize)
plt.xticks(fontsize=ticksize)
plt.yticks(fontsize=ticksize)
plt.savefig(path+'strain.png')
plt.tight_layout()
plt.show()

In [ ]:
# Qubits for templates
Mqubits = 17
# Qubits for Grover's
Pqubits = 11

# Equivalent number of states
M = 2**Mqubits
P = 2**Pqubits

In [ ]:
tag = str(M)+'_'+str(P)+'_'+str(SNR_threshold).replace('.','_')+'_'+tag

First let us make the state:

$$ |\psi_{\text{ini}}⟩ = \frac{1}{\sqrt{N}}\sum^{N-1}_{i=0}|i⟩|0⟩^{\times M}. $$

This state is what the template information will be stored in. Let us also define:

$$ |I⟩ = \frac{1}{\sqrt{M}}\sum^{M-1}_{i=0}|i⟩. $$

This state has a basis state that corresponds to the indexes of each template. We perform operation $\hat{k}_{1}$ on this state to give a state that represent the template waveforms in the frequency domanin $|T⟩$:

$$ |I⟩\otimes |T⟩ = \hat{k}_{1}(|I⟩\otimes |\psi_{\text{ini}}⟩). $$

Similarly the data is loaded into a state represented by $|D⟩$.

In [ ]:
index_states = np.ones(M).astype(dtype)/np.sqrt(M)

Grover's algorithm is applied after matched filtering to all templates. On a quantum computer, this can be done in parallel to give an equal amplitude state $|w⟩$ of length $N$ but any state that corresponds to the index of a template that meets the criteria of $\rho>\rho_{\text{thr}}$ has a phase-flip of -1. $|w⟩$ is made by applying $\hat{k}_{2}$:

$$|I⟩\otimes|T⟩\otimes|D⟩\otimes|w⟩ = \hat{k}_{2}(|I⟩\otimes|T⟩\otimes|D⟩\otimes|0⟩^{\times N}).$$

In [ ]:
if load_states and os.path.isfile(path+'snrs_'+tag+'.npy'):
    print('Loading SNR')
    snrs = np.load(path+'snrs_'+tag+'.npy')
    w = np.where(snrs>=SNR_threshold,-1.,1.)/np.sqrt(M)
elif load_states and os.path.isfile('data/SNRs_signal_spins.npy'):
    print('Loading SNR (from data/SNRs_signal_spins.npy)')
    snrs_ = np.load('data/SNRs_signal_spins.npy')
    if len(snrs_)>M:
        snrs = snrs_[::len(snrs_)//M]
        if len(snrs)>M:
            snrs = snrs[:M]
        elif len(snrs)<M:
            snrs__ = snrs_[1:][::len(snrs)//M][:M-len(snrs)]
            snrs = np.concatenate((snrs,snrs__))
    else:
        snrs = snrs_
    w = np.where(snrs>=SNR_threshold,-1.,1.)/np.sqrt(M)

else:
    print('Calculating SNR')
    # Apply k1 and k2 to get w states
    w, snrs = qmffn.k_12(index_states, Data, psd, threshold=SNR_threshold, bankfunc=bankfunc, temp_file=temp_file, spins=spins, cores=cores)
    
# Save states
if save_states:
    np.save(path+'/snrs_'+tag,snrs)

In [ ]:
M = len(w)

The first part of Grover's algorithm is then applied as follows:

1. Create operator $\hat{U}_{w}=\mathcal{I}-2|w⟩⟨w|$ where $w$ is the matrix position corresponding to the matching templates. This operator has the property:

$$ \hat{U}_{w} |x⟩ = -|x⟩ \text{ if } x=w, $$
$$ \hat{U}_{w} |x⟩ = |x⟩ \text{ if } x\ne w.\ \ \$$

2. Initiate superposition:

$$ |s⟩ = \frac{1}{\sqrt{N}}\sum^{N-1}_{x=0}|x⟩, $$

assuming that every template is equally likely to have the correct template without any more prior knowledge.

3. Create the Grover diffusion operator $\hat{U}_{s}=2|s⟩⟨s|-\mathcal{I}$.

4. Apply $\hat{U}_{w}$ then $\hat{U}_{s}$ to $|s⟩$ $p$ times to each state in $P$ ($p=\{0,1,...P-1\}$).

Now we apply Grover's algorithm itterably to this state such that:

$$ |\psi_{1}⟩ = \hat{C}_{G}|\psi_{0}⟩, $$

where $\hat{C}_{G}|p⟩\otimes |x⟩\rightarrow |p⟩\otimes (\hat{G})^{p}|x⟩$,

and $\hat{G}=\hat{U_{s}}\hat{U_{w}}$.

First we make the state:

$$ |\psi_{0}⟩ = \frac{1}{\sqrt{PN}}\sum^{P-1}_{p=0}\sum^{N-1}_{i=0}|p⟩|i⟩. $$


In [ ]:
if load_states and os.path.isfile(path+'psi1_in_'+tag+'.npy'):
    print('Loading psi1...')
    psi_ = np.load(path+'psi1_in_'+tag+'.npy')
    P,M = psi_.shape
else:
    # Apply the first step to quantum counting
    psi_ = qmffn.quantum_counting1(w,np.ones((M,P)).astype(dtype)/np.sqrt(M*P), dtype=dtype)

# Save states
if save_states:
    np.save(path+'psi1_in_'+tag,psi_)

In [ ]:
plot_psi_1_paper.main([path+'/psi1_in_'+tag+'.npy'], outpath=path)

### Quantum Fourier Transform

The *quantum fourier transform* is much the same as it's classical counterpart but is performed on amplitude/phase information stored on the states of qubits. It transfers information stored in amplitudes in quantum states into phase information. There also exists the inverse quantum fourier transform for the reverse opperation.

The quantum fourier transform acting on state $|p⟩$ gives:

$$ \mathrm{QFT}:|x⟩ \mapsto \frac{1}{\sqrt{K}}\sum^{K-1}_{k=0}e^{2\pi i\frac{kx}{K}}|k⟩. $$

The inverse quantum fourier transform is applied across the ancillary qubits the recover the phase information from the sinusoidal behaviour in the states shown in the graph above. This requires creating a $QFT^{-1}$ operator of size $P\times P$, which we will call $\hat{F}_{P}$. Applying this across the ancillary states in $|\psi_{1}⟩$:

$$ \psi_{2} = \hat{F}_{P}\psi_{1}. $$

In [ ]:
if load_states and os.path.isfile(path+'psi2_in_'+tag+'.npy'):
    print('Loading psi2...')
    psi_ = None
    psi = np.load(path+'psi2_in_'+tag+'.npy')
    P,M = psi.shape
else:
    # Apply the second step to quantum counting
    psi_ = qmffn.quantum_counting2(psi_)
    psi = psi_
    psi_ = None
    
# Save states
if save_states:
    np.save(path+'psi2_in_'+tag,psi)

In [ ]:
plot_psi_2_paper.main([path+'/psi2_in_'+tag+'.npy'], outpath=path)

In [ ]:
plot_psi_2_k_paper.main([path+'/psi2_in_'+tag+'.npy'], outpath=path)

In [ ]:
plot_psi_2_t_paper.main([path+'/psi2_in_'+tag+'.npy'], outpath=path)

In [ ]:
qmfsim.main(path+'psi2_in_'+tag+'.npy', path, tag, all_trials=[0,1], runs=1000)

In [ ]:
plot_simulation.main(path+'simulation_out_0_'+tag+'.npy', path+'simulation_out_1_'+tag+'.npy', path, P, N, M)